# Scraping Data from Used-Cars listing Website

### Project Goal-
    - To extract maximum possible information on used cars in major cities in india.
    - First phase is to gather all vehicles links listed by cities name
    - Second phase is to load each link obtained in 1st phase and gather all information available about those vehicles.
    - Then finally export data to Excel file

## Used Cars listing Pages

In [68]:
# These are the links of Used-Cars listing web-pages referring to different cities
# In further section I have defined a function code which will go through the link provided, scroll it, and gather all links.
cities_links=['https://www.cardekho.com/used-cars+in+ahmedabad',
              'https://www.cardekho.com/used-cars+in+bangalore',
              'https://www.cardekho.com/used-cars+in+chennai',
              'https://www.cardekho.com/used-cars+in+delhi-ncr',
              'https://www.cardekho.com/used-cars+in+hyderabad',
              'https://www.cardekho.com/used-cars+in+jaipur',
              'https://www.cardekho.com/used-cars+in+kolkata',
              'https://www.cardekho.com/used-cars+in+mumbai',
              'https://www.cardekho.com/used-cars+in+pune'
             ]

## Getting all links within listing pages

In [70]:
# Here I imported the necessary modules requires for web scraping and other functions
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import time
from time import sleep
import pandas as pd
from numpy import random

s = Service(r"C:\Users\hp\Documents\chromedriver.exe") # created service object for accessing chrome driver
car_links=[]     #These two lists will save all used cars links obtained and will be stored on dataframe
CarLocation=[]

# Below is the function which takes input 'city-name' and 'link' of car-listing page and stores all used-cars links within that page
# I've defined a function because when tried in loop, it gets irregular numbers of links
def get_all_links(city, city_link):
    global current_links
    current_links=[]
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    driver.get(city_link)
    time.sleep(3)

    element= driver.find_element(By.TAG_NAME,"body")
    # As the website detects bots by regular pattern in activity, and stops sending data, So i have tried to keep it random by using below loop for scrolling
    for j in range(0,350):              
        if j in (20,21,50,51,70,71,95,99,140,145,190,195,250,254,290,299,312,313,315,317,318):
            element.send_keys(Keys.PAGE_UP)
        else:    
            element.send_keys(Keys.PAGE_DOWN)
        time.sleep(random.randint(2,6))

    links = driver.find_elements(By.XPATH,"//h3[@class='title']//a")
    for link in links:
        current_links.append(link.get_attribute('href'))
    if len(current_links)>=900:            #So as to get maxmimum links out of one page, if links are lesser, it wont add up, we will try again
        car_links.extend(current_links)
        for l in range(len(current_links)):    #add respective city name
            CarLocation.append(city)
    time.sleep(random.randint(5))
    driver.close()
    print('Current results:',len(current_links))
    print('Total Car links-',len(car_links))

In [71]:
# Ahmedabad city cars
get_all_links('Ahmedabad','https://www.cardekho.com/used-cars+in+ahmedabad')

Current results: 1000
Total Car links- 1000


In [72]:
# Bangalore city cars
get_all_links('Bangalore','https://www.cardekho.com/used-cars+in+bangalore')

Current results: 1040
Total Car links- 2040


In [73]:
# Chennai city cars
get_all_links('Chennai','https://www.cardekho.com/used-cars+in+chennai')

Current results: 1000
Total Car links- 3040


In [74]:
# Delhi city cars
get_all_links('Delhi','https://www.cardekho.com/used-cars+in+delhi-ncr')

Current results: 1160
Total Car links- 4200


In [75]:
# Hyderabad city cars
get_all_links('Hyderabad','https://www.cardekho.com/used-cars+in+hyderabad')

Current results: 1000
Total Car links- 5200


In [77]:
# Jaipur city cars
get_all_links('Jaipur','https://www.cardekho.com/used-cars+in+jaipur')

Current results: 1020
Total Car links- 6220


In [78]:
# Kolkata city cars
get_all_links('Kolkata','https://www.cardekho.com/used-cars+in+kolkata')

Current results: 1000
Total Car links- 7220


In [79]:
# Mumbai city cars
get_all_links('Mumbai','https://www.cardekho.com/used-cars+in+mumbai')

Current results: 1100
Total Car links- 8320


In [80]:
# Pune city cars
get_all_links('Pune','https://www.cardekho.com/used-cars+in+pune')

Current results: 1000
Total Car links- 9320


### Store all links in a DataFrame and back-up in excel

In [81]:
temp_df=pd.DataFrame(list(zip(CarLocation,car_links)),columns=['Car_location','car_link'])
temp_df.to_excel(r"C:\Users\hp\Documents\final_links_if_all_run2.xlsx")

In [89]:
pd.set_option('display.max_columns', None)    #show all columns in dataframe
temp_df.head()

,Car_location,car_link
0,Ahmedabad,https://www.cardekho.com/used-car-details/used...
1,Ahmedabad,https://www.cardekho.com/used-car-details/used...
2,Ahmedabad,https://www.cardekho.com/used-car-details/used...
3,Ahmedabad,https://www.cardekho.com/used-car-details/used...
4,Ahmedabad,https://www.cardekho.com/used-car-details/used...


### List of all Used-Cars links for further data extraction 

In [107]:
df_links= pd.read_excel(r"C:\Users\hp\Documents\final_links_if_all_run2.xlsx")
links=list(df_links.car_link)
city=list(df_links.Car_location)

## Scraping/Gathering data within each page
    following block of code will perform these actions:
        -Initialize empty lists
        -Looping through each page of used cars stored in earlier section
        -Pass a fake user-id for different pages for not getting blocked
        -Try-Except block for handeling exceptions and get data as accurate as possible and ignore faulty data
        -Clicking dropdown elements 
        -Adding data to respective lists and then create a final DataFrame

In [149]:
# initialize multiple empty lists
VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price= ([] for i in range(19))
ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat=([] for i in range(10))
EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger=([] for i in range(10))
BodyLength,BodyWidth,BodyHeight,WheelBase=([] for i in range(4))
GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers=([] for i in range(7))

import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent          # Library for generating fake-user agents

s = Service(r"C:\Users\hp\Documents\chromedriver.exe")

In [150]:
# Here i created callable looping data extraction function , so that if process gets interrupted, we can always resume.
def GetInsideData(links,city): 
    for l in range(len(links)):
        try:
            ua = UserAgent()
            userAgent= ua.ie
            options= Options()
            options.add_argument(f'user-agent={userAgent}')
            driver = webdriver.Chrome(options=options, service=s)

            driver.maximize_window()
            driver.get(links[l])
            time.sleep(random.randint(3,5))

            name= driver.find_element(By.XPATH, "//div[@class='vehicleName']")
            VehicleName.append(name.text)

            CarLocation.append(city[l])

            car_price=driver.find_element(By.XPATH,"//div[@class='vehiclePrice']")
            Price.append(car_price.text)

            x= driver.find_elements(By.XPATH, "//div[@class='outer-card-container posR']//span[@class='value-text ']")
            RegistrationYear.append(x[0].text)
            Insurance.append(x[1].text)
            FuelType.append(x[2].text)
            Seats.append(x[3].text)
            KmsDriven.append(x[4].text)
            RTO.append(x[5].text)
            Ownership.append(x[6].text)
            Engine.append(x[7].text)
            TransmissionType.append(x[8].text)
            YearOfManufacture.append(x[9].text)

            # This block of code will get info from any of the two places, where data is present (beacuse website is dynamic)
            try:
                on_road= driver.find_element(By.XPATH, "//span[@class='pricerangeNC']")
            except Exception as e:
                pass
            try:
                on_road= driver.find_element(By.XPATH, "//div[@id='fair2-New-Car-On-Road-Price']//div[@class='price-info-amt']")
            except Exception as e:
                pass
            OnRoadPrice.append(on_road.text)

            features= driver.find_elements(By.XPATH, "//ul[@class='gsc_row']//span")
            feat=[]
            for f in features:
                feat.append(f.text)
            VehFeatures.append(feat)

            # This is the Drop-Down section code, it will click dropdown lists within 2 different dropdown sections
            driver.find_element(By.XPATH,"//div[@class='outer-card-container carFeaturesContainer']//div[@class='cta-text turn-downside']").click()
            time.sleep(2)
            all_feat=driver.find_elements(By.XPATH,"//div[@class='accordianSec']//div[@class='featureList']//div[@class='borderBottom features-accordian accordionOff  ']")
            for fe in range(len(all_feat)):
                if all_feat[fe].is_displayed():
                    try:
                        all_feat[fe-1].click()
                        time.sleep(random.randint(2,5))
                    except Exception as e:
                        print(e)
                        pass

            driver.find_element(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='cta-text turn-downside']").click()
            time.sleep(2)
            all_spec=driver.find_elements(By.XPATH,"//div[@class='gsc_row detailsList']//h3")
            for sp in range(len(all_spec)):
                if all_spec[sp].is_displayed():
                    try:
                        all_spec[sp-1].click()
                        time.sleep(random.randint(2,5))
                    except Exception as e:
                        print(e)
                        pass

            # extra_features within features drop-down 
            comfort_feat=driver.find_elements(By.XPATH, "//div[@class='outer-card-container carFeaturesContainer']//div[@class='featureList'][1]//div[@class='content  on']//span[@class='value-text ']")
            interior_feat=driver.find_elements(By.XPATH, "//div[@class='outer-card-container carFeaturesContainer']//div[@class='featureList'][2]//div[@class='content  on']//span[@class='value-text ']")
            exterior_feat=driver.find_elements(By.XPATH, "//div[@class='outer-card-container carFeaturesContainer']//div[@class='featureList'][3]//div[@class='content  on']//span[@class='value-text ']")
            safety_feat=driver.find_elements(By.XPATH, "//div[@class='outer-card-container carFeaturesContainer']//div[@class='featureList'][4]//div[@class='content  on']//span[@class='value-text ']")
            EntAndCommunication_feat=driver.find_elements(By.XPATH, "//div[@class='outer-card-container carFeaturesContainer']//div[@class='featureList'][5]//div[@class='content  on']//span[@class='value-text ']")

            # Defined a function for data inside extra feature dropdown sections
            def UploadFeatList(FeatElement,f_list,len_feat):
                temp_list=[]
                for fe in FeatElement:
                    temp_list.append(fe.text)
                f_list.append(temp_list)
                len_feat.append(len(temp_list))

            UploadFeatList(comfort_feat,ComfortFeatures,TotalComFeat)
            UploadFeatList(interior_feat,InteriorFeatures,TotalIntFeat)
            UploadFeatList(exterior_feat,ExteriorFeatures,TotalExtFeat)
            UploadFeatList(safety_feat,SafetyFeatures,TotalSafeFeat)
            UploadFeatList(EntAndCommunication_feat,EntAndCommFeatures,TotalEntAndCommFeat)  

            #Specifications-
            specs = driver.find_elements(By.XPATH, "//div[@id='vdpCard-carFeatures']//ul[@class='gsc_row detailsList']//span")
            Mileage.append(specs[0].text)
            MaxPower.append(specs[2].text)
            Torque.append(specs[3].text)
            WheelSize.append(specs[4].text)

            #Extra specs- (Get data only if it is present w.r.t tags)
            EngTrTag=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][1]//div[@class='label ']")
            EngTrEle=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][1]//span[@class='value-text ']")
            for et in range(len(EngTrTag)):
                if EngTrTag[et].text=='Color':
                    EngineColor.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Engine Type':
                    EngineType.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Displacement':
                    Displacement.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Max Power':
                    MaxPower.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Max Torque':
                    MaxTorque.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='No of Cylinder':
                    NumOfCylinder.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Values per Cylinder':
                    ValuesPerCylinder.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Fuel Suppy System':
                    FuelSupplySystem.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Turbo Charger':
                    TurboCharger.append(EngTrEle[et].text)
                elif EngTrTag[et].text=='Super Charger':
                    SuperCharger.append(EngTrEle[et].text)
                else:
                    pass

            DimensionTag=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][2]//div[@class='label ']")
            DimensionEle=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][2]//span[@class='value-text ']")
            for dm in range(len(DimensionTag)):
                if DimensionTag[dm].text=='Length':
                    BodyLength.append(DimensionEle[dm].text)
                elif DimensionTag[dm].text=='Width':
                    BodyWidth.append(DimensionEle[dm].text)
                elif DimensionTag[dm].text=='Height':
                    BodyHeight.append(DimensionEle[dm].text)
                elif DimensionTag[dm].text=='Wheel Base':
                    WheelBase.append(DimensionEle[dm].text)
                else:
                    pass

            misslTag=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][3]//div[@class='label ']")
            misslEle=driver.find_elements(By.XPATH,"//div[@class='outer-card-container specsCard']//div[@class='gsc_row detailsList'][3]//span[@class='value-text ']")
            for m in range(len(misslTag)):
                if misslTag[m].text=="Gear Box":
                    GearBox.append(misslEle[m].text)
                elif misslTag[m].text=="Seating Capacity":
                    SeatingCapacity.append(misslEle[m].text)
                elif misslTag[m].text=="Steering Type":
                    SteeringType.append(misslEle[m].text)
                elif misslTag[m].text=="Front Brake Type":
                    FrontBrakeType.append(misslEle[m].text)
                elif misslTag[m].text=="Rear Brake Type":
                    RearBrakeType.append(misslEle[m].text)
                elif misslTag[m].text=="Tyre Type":
                    TyreType.append(misslEle[m].text)
                elif misslTag[m].text=="No Door Numbers":
                    DoorNumbers.append(misslEle[m].text)
                else:
                    pass
            time.sleep(3)
            driver.close()
        except Exception as e:
            print(e)           # if any error occurs, it will simply ignore that page & print exception & move on to next page
            driver.close()

### Calling scraping function over all links:- Attempt 1

In [108]:
GetInsideData(links,city)    # Calling function over all stored links & city lists (defined in previous block)   
time.sleep(2)
df2= pd.DataFrame(list(zip(VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers)),
                  columns=['VehicleName','CarLocation','RegistrationYear','Insurance','FuelType','Seats','KmsDriven','RTO','Ownership','Engine','TransmissionType','YearOfManufacture','VehFeatures','Mileage','MaxPower','Torque','WheelSize','OnRoadPrice','Price','ComfortFeatures','TotalComFeat','InteriorFeatures','TotalIntFeat','ExteriorFeatures','TotalExtFeat','SafetyFeatures','TotalSafeFeat','EntAndCommFeatures','TotalEntAndCommFeat','EngineColor','EngineType','Displacement','MaxPower','MaxTorque','NumOfCylinder','ValuesPerCylinder','FuelSupplySystem','TurboCharger','SuperCharger','BodyLength','BodyWidth','BodyHeight','WheelBase','GearBox','SeatingCapacity','SteeringType','FrontBrakeType','RearBrakeType','TyreType','DoorNumbers'])
df2

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
Message: element not interactable
 

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No s

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No s

list index out of range
Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No sym

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No s

...

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

,VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers
0,2019 Maruti Ciaz Alpha Automatic BSIV,Ahmedabad,Feb 2019,Comprehensive,Petrol,5 Seats,"56,000 Kms",GJ38,First Owner,1462 cc,Automatic,2019,"[Power Steering, Power Windows Front, Air Cond...",20.28 kmpl,103.25bhp,138Nm,16,7.71-11.98 Lakh*,₹ 8.50 Lakh,"[Power Steering, Power Windows Front, Power Wi...",22,"[Air Conditioner, Heater, Adjustable Steering,...",12,"[Adjustable Head Lights, Fog Lights Front, Pow...",11,"[Anti Lock Braking System, Centeral Locking, P...",29,"[Radio, Speakers Front, Speakers Rear, Integra...",8,Silver,k15B VVT Engine,1462,103.25bhp,138Nm@4400rpm,4,4,MPFI,No,No,4490 mm,1730 mm,1485 mm,2650 mm,4 Speed,5,Power,Ventilated Disc,Drum,"Tubeless, Radial",4
1,2012 Honda Brio S MT,Ahmedabad,Sept 2012,Third Party,Petrol,5 Seats,"80,963 Kms",GJ01,First Owner,1198 cc,Manual,2012,"[Power Steering, Power Windows Front, Air Cond...",19.4 kmpl,103.25bhp@6000rpm,109Nm,5,4.25-6.09 Lakh*,₹ 2.50 Lakh,"[Power Steering, Power Windows Front, Power Wi...",10,"[Air Conditioner, Heater, Digital Odometer, Ta...",8,"[Adjustable Head Lights, Power Adjustable Exte...",5,"[Centeral Locking, Power Door Locks, Child Saf...",16,"[Radio, Speakers Front, Speakers Rear]",3,Energetic Blue - Brio,i-VTEC Engine,1198,103.25bhp@6000rpm,109Nm@4500rpm,4,4,PGM - Fi,No,No,3610mm,1680mm,1500mm,2345mm,5 Speed,5,Power,Disc,Drum,"Tubeless,Radial",5
2,2010 Hyundai i10 Era,Ahmedabad,Apr 2010,Third Party,Petrol,5 Seats,"92,585 Kms",GJ01,Second Owner,1086 cc,Manual,2010,"[Power Steering, Remote Trunk Opener, Air Cond...",19.81 kmpl,86.8bhp,99.04Nm,5,3.79-6.55 Lakh*,₹ 2 Lakh,"[Power Steering, Remote Trunk Opener, Remote F...",6,"[Air Conditioner, Heater, Digital Odometer, Ta...",7,"[Adjustable Head Lights, Manually Adjustable E...",4,"[Child Safety Locks, Passenger Side Rear View ...",11,[],0,Sleek Silver,IRDE2 Engine,1086,86.8bhp,99.04Nm@4500rpm,4,3,MPFI,No,No,3585mm,1595mm,1550mm,2380mm,5 Speed,5,Power,Ventilated Disc,Drum,Tubeless,5
3,2010 Hyundai i10 Era,Ahmedabad,Apr 2010,Comprehensive,Petrol,5 Seats,"64,675 Kms",PB03,First Owner,1086 cc,Manual,2010,"[Power Steering, Remote Trunk Opener, Air Cond...",19.81 kmpl,86.8bhp@6000rpm,99.04Nm,5,3.79-6.55 Lakh*,₹ 1.60 Lakh,"[Power Steering, Remote Trunk Opener, Remote F...",6,"[Air Conditioner, Heater, Digital Odometer, Ta...",7,"[Adjustable Head Lights, Manually Adjustable E...",4,"[Child Safety Locks, Passenger Side Rear View ...",11,[],0,White,IRDE2 Engine,1086,86.8bhp@6000rpm,99.04Nm@4500rpm,4,3,MPFI,No,No,3585mm,1595mm,1550mm,2380mm,5 Speed,5,Power,Ventilated Disc,Drum,Tubeless,5
4,2018 Maruti Baleno 1.2 Zeta,Ahmedabad,Jan 2018,Not Available,Petrol,5 Seats,"29,869 Kms",GJ01,First Owner,1197 cc,Manual,2018,"[Power Steering, Power Windows Front, Air Cond...",21.4 kmpl,68.05bhp,115Nm,16,5.90-9.66 Lakh*,₹ 7.50 Lakh,"[Power Steering, Power Windows Front, Power Wi...",13,[],0,"[Adjustable Head Lights, Fog Lights Front, Pow...",13,"[Anti Lock Braking System, Centeral Locking, P...",27,"[Cd Player, Radio, Speakers Front, Speakers Re...",8,Autumn Orange,VVT Petrol Engine,1197,68.05bhp,115Nm@4000rpm,4,4,MPFI,No,No,3995mm,1745mm,1510mm,2520mm,5 Speed,5,Power,Disc,Drum,"Tubeless,Radial",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6346,2018 Tata Nexon 1.2 Re

## Info & Back-up data

In [111]:
df2.info()
df2.to_excel(r"C:\Users\hp\Documents\Used_cars_data.xlsx")    #store into excel file

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6351 entries, 0 to 6350
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   VehicleName          6351 non-null   object
 1   CarLocation          6351 non-null   object
 2   RegistrationYear     6351 non-null   object
 3   Insurance            6351 non-null   object
 4   FuelType             6351 non-null   object
 5   Seats                6351 non-null   object
 6   KmsDriven            6351 non-null   object
 7   RTO                  6351 non-null   object
 8   Ownership            6351 non-null   object
 9   Engine               6351 non-null   object
 10  TransmissionType     6351 non-null   object
 11  YearOfManufacture    6351 non-null   object
 12  VehFeatures          6351 non-null   object
 13  Mileage              6351 non-null   object
 14  MaxPower             6351 non-null   object
 15  Torque               6351 non-null   object
 16  WheelS

> Dataframe contains 50 columns, 6351 non-null records. Let's see until which city, process was successful, and when did it broke?

In [142]:
df2.tail(2)

,VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers
6349,2012 Audi Q5 2.0 TDI,Kolkata,Nov 2012,Third Party,Diesel,5 Seats,"40,489 Kms",WB02,First Owner,1086 cc,Automatic,2013,"[Power Steering, Power Windows Front, Air Cond...",12.7 kmpl,82bhp@6000rpm,420Nm,18,7.41-10.02 Lakh*,₹ 2.50 Lakh,"[Power Steering, Power Windows Front, Power Wi...",20,"[Air Conditioner, Heater, Adjustable Steering,...",12,"[Adjustable Head Lights, Fog Lights Front, Pow...",13,"[Anti Lock Braking System, Brake Assist, Cente...",29,"[Cd Player, Radio, Audio System Remote Control...",5,Maroon,1.2L Kappa Dual VTVT Petrol,3198,82bhp@6000rpm,470Nm@1750-2500rpm,4,4,MPFI,No,No,3990mm,1845mm,1505mm,2873 mm,6-Speed,5,Power,Drum,Drum,"Tubeless,Radial",4
6350,2014 Ford Ecosport 1.5 DV5 MT Trend,Kolkata,Dec 2014,Comprehensive,Diesel,5 Seats,"50,000 Kms",WB02,First Owner,1197 cc,Automatic,2016,"[Power Steering, Power Windows Front, Air Cond...",31.79 kmpl,261.50bhp,78Nm,16,30.28-41.70 Lakh*,₹ 3.99 Lakh,"[Power Steering, Power Windows Front, Power Wi...",10,[],0,"[Adjustable Head Lights, Manually Adjustable E...",6,"[Anti Lock Braking System, Centeral Locking, C...",18,[],0,Maroon,M 264 petrol engine,1198,261.50bhp,170Nm@1750-4000rpm,4,4,MPFI,Yes,No,3595mm,1735mm,1700mm,2550mm,6 Speed,5,Power,Ventilated Disc,Drum,"Radial, Tubless",4


> Here we can see that loop is broken in city kolkata links, so lets try again for links of 2 cities which are left i.e, Mumbai & Pune. Although I have backup of previous data, so that it wont get lost.

### Getting Data from Remaining links - Attempt 2

In [132]:
remaining_cities=['Mumbai','Pune']
remaining_df=df_links[df_links['Car_location'].isin(remaining_cities)]
rem_links=list(remaining_df.car_link)
rem_city=list(remaining_df.Car_location)

In [138]:
GetInsideData(rem_links,rem_city)         #calling function over remaining links

0 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

1 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)

153 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

156 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.

222 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='outer-card-container carFeaturesContainer']//div[@class='cta-text turn-downside']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

223 Message: no such element: Unable to locate element: {"method":"xpath","selector

351 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

368 list index out of range
378 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Sess

430 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

431 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.

...

1835 Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

1841 list index out of range
1844 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Back

1910 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

1912 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.535

1932 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

1940 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.535

1981 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

1983 Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807F

2060 Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

2062 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F2

In [143]:
# Store result to different dataframe-
df3= pd.DataFrame(list(zip(VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers)),
                  columns=['VehicleName','CarLocation','RegistrationYear','Insurance','FuelType','Seats','KmsDriven','RTO','Ownership','Engine','TransmissionType','YearOfManufacture','VehFeatures','Mileage','MaxPower','Torque','WheelSize','OnRoadPrice','Price','ComfortFeatures','TotalComFeat','InteriorFeatures','TotalIntFeat','ExteriorFeatures','TotalExtFeat','SafetyFeatures','TotalSafeFeat','EntAndCommFeatures','TotalEntAndCommFeat','EngineColor','EngineType','Displacement','MaxPower','MaxTorque','NumOfCylinder','ValuesPerCylinder','FuelSupplySystem','TurboCharger','SuperCharger','BodyLength','BodyWidth','BodyHeight','WheelBase','GearBox','SeatingCapacity','SteeringType','FrontBrakeType','RearBrakeType','TyreType','DoorNumbers'])
df3.tail(3)    #See if we got all data this time?

,VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers
7737,2017 Maruti Ignis 1.2 Zeta BSIV,Mumbai,Third Party insurance,Petrol,5 Seats,"25,000 Kms",Second Owner,1197 cc,First Owner,1598 cc,Automatic,2014,"[Power Steering, Power Windows Front, Air Cond...",16.1 kmpl,74.02bhp,11.5,17,7.41-10.02 Lakh*,₹ 4.20 Lakh,"[Power Steering, Power Windows Front, Power Wi...",11,[],0,"[Adjustable Head Lights, Fog Lights Front, Fog...",7,"[Anti Lock Braking System, Brake Assist, Cente...",15,[],0,White,K4M Petrol Engine,1956,74.02bhp,350nm@1750-2500rpm,4,4,MPFI,Yes,No,3995mm,1760mm,1515mm,2807mm,5 Speed,5,Electric,Ventilated Disc,Disc,"Tubeless,Radial",4
7738,2021 Hyundai Creta EX Diesel,Mumbai,Jun 2021,Zero Dep,Diesel,5 Seats,"18,031 Kms",GJ01,First Owner,1598 cc,Automatic,2015,"[Power Steering, Power Windows Front, Air Cond...",28 km/kg,74.02bhp@4000rpm,95.2nm,16,6.92-8.82 Lakh*,₹ 11.85 Lakh,"[Power Steering, Power Windows Front, Power Wi...",15,[],0,"[Adjustable Head Lights, Power Adjustable Exte...",9,"[Anti Lock Braking System, Centeral Locking, P...",23,"[Radio, Speakers Front, Speakers Rear, Integra...",7,Silver,Kappa VTVT Petrol Engine,1197,74.02bhp@4000rpm,114.73Nm@4000rpm,3,4,MPFI,Yes,No,4490mm,1735,1485mm,2808mm,5 Speed,5,Power,Disc,Drum,"Tubeless,Radials",5
7739,2008 Maruti SX4 Zxi BSIII,Mumbai,Aug 2008,Third Party insurance,Petrol,5 Seats,"79,100 Kms",MH04,First Owner,1968 cc,Automatic,2011,"[Power Steering, Power Windows Front, Air Cond...",17.9 kmpl,91.1bhp,500Nm,17,28.18-36.88 Lakh*,₹ 1.50 Lakh,"[Power Steering, Power Windows Front, Power Wi...",30,"[Air Conditioner, Heater, Adjustable Steering,...",16,"[Adjustable Head Lights, Power Adjustable Exte...",19,"[Anti Lock Braking System, Brake Assist, Cente...",32,"[Cd Player, Radio, Speakers Front, Speakers Re...",8,White,1.0L TSI,1997,91.1bhp,280Nm@1800-2800rpm,4,4,MPFI,Yes,No,3995mm,1660mm,1555mm,2469mm,4 Speed,5,Power,Disc,Drum,"Tubeless,Radial",4


> So previous loop also stopped before Pune city links, lets get them-

In [157]:
remained_df=df_links[df_links['Car_location']=='Pune']
Pune_links=list(remained_df.car_link)
Pune_city=list(remained_df.Car_location)
print(len(Pune_links),len(Pune_city))

1000 1000


In [158]:
GetInsideData(Pune_links,Pune_city)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No s

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='outer-card-container carFeaturesContainer']//div[@class='cta-text turn-downside']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

...

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FD04D]
	(No symbol) [0x0072C0B0]
	(No symbol) [0x0072C22B]
	(No symbol) [0x0075E612]
	(No symbol) [0x007485D4]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Sta

Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No symbol) [0x00807FD1]
	(No symbol) [0x006FCF10]
	(No symbol) [0x0072D003]
	(No symbol) [0x007232A6]
	(No symbol) [0x0074858C]
	(No symbol) [0x00722BFF]
	(No symbol) [0x00748804]
	(No symbol) [0x0075C9EB]
	(No symbol) [0x00748386]
	(No symbol) [0x0072163C]
	(No symbol) [0x0072269D]
	GetHandleVerifier [0x00B19A22+2655074]
	GetHandleVerifier [0x00B0CA24+2601828]
	GetHandleVerifier [0x00928C0A+619850]
	GetHandleVerifier [0x00927830+614768]
	(No symbol) [0x008105FC]
	(No symbol) [0x00815968]
	(No symbol) [0x00815A55]
	(No symbol) [0x0082051B]
	BaseThreadInitThunk [0x76BD6BD9+25]
	RtlGetFullPathName_UEx [0x775D8FD2+1218]
	RtlGetFullPathName_UEx [0x775D8F9D+1165]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='vehicleName']"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0087F243]
	(No s

In [160]:
# Store result to different dataframe-
df4= pd.DataFrame(list(zip(VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers)),
                  columns=['VehicleName','CarLocation','RegistrationYear','Insurance','FuelType','Seats','KmsDriven','RTO','Ownership','Engine','TransmissionType','YearOfManufacture','VehFeatures','Mileage','MaxPower','Torque','WheelSize','OnRoadPrice','Price','ComfortFeatures','TotalComFeat','InteriorFeatures','TotalIntFeat','ExteriorFeatures','TotalExtFeat','SafetyFeatures','TotalSafeFeat','EntAndCommFeatures','TotalEntAndCommFeat','EngineColor','EngineType','Displacement','MaxPower','MaxTorque','NumOfCylinder','ValuesPerCylinder','FuelSupplySystem','TurboCharger','SuperCharger','BodyLength','BodyWidth','BodyHeight','WheelBase','GearBox','SeatingCapacity','SteeringType','FrontBrakeType','RearBrakeType','TyreType','DoorNumbers'])

## Final DataFrame

In [166]:
# Here lets add previous records with new ones and create final df to be saved as excel file
final_df=pd.concat([df3,df4],axis=0,ignore_index=True)
final_df.tail(2)

,VehicleName,CarLocation,RegistrationYear,Insurance,FuelType,Seats,KmsDriven,RTO,Ownership,Engine,TransmissionType,YearOfManufacture,VehFeatures,Mileage,MaxPower,Torque,WheelSize,OnRoadPrice,Price,ComfortFeatures,TotalComFeat,InteriorFeatures,TotalIntFeat,ExteriorFeatures,TotalExtFeat,SafetyFeatures,TotalSafeFeat,EntAndCommFeatures,TotalEntAndCommFeat,EngineColor,EngineType,Displacement,MaxPower,MaxTorque,NumOfCylinder,ValuesPerCylinder,FuelSupplySystem,TurboCharger,SuperCharger,BodyLength,BodyWidth,BodyHeight,WheelBase,GearBox,SeatingCapacity,SteeringType,FrontBrakeType,RearBrakeType,TyreType,DoorNumbers
8395,2013 Nissan Sunny Diesel XL,Pune,Dec 2013,Comprehensive,Diesel,5 Seats,"62,000 Kms",MH42,First Owner,999 cc,Manual,2016,"[Power Steering, Power Windows Front, Air Cond...",18.6 kmpl,74bhp,114Nm,5,2.60-3.80 Lakh*,₹ 2.99 Lakh,"[Power Steering, Power Windows Front, Power Wi...",13,[],0,"[Adjustable Head Lights, Fog Lights Front, Fog...",11,"[Anti Lock Braking System, Brake Assist, Cente...",20,"[Cd Player, Radio, Speakers Front, Speakers Rear]",4,Silver,Kappa VTVT Petrol Engine,998,74bhp,90Nm@3500rpm,4,4,MPFI,No,No,3595mm,1495mm,1700mm,2360mm,5 Speed,5,Power,Ventilated Disc,Drum,Tubeless Tyres,4
8396,2011 Skoda New Laura 1.8 TSI Ambiente,Pune,Dec 2011,Comprehensive,Petrol,5 Seats,"80,000 Kms",MH43,Second Owner,1198 cc,Manual,2016,"[Power Steering, Power Windows Front, Air Cond...",31.79 kmpl,74bhp@4000rpm,78Nm,14,3.29-6.58 Lakh*,₹ 2.49 Lakh,"[Power Steering, Power Windows Front, Power Wi...",10,[],0,"[Adjustable Head Lights, Manually Adjustable E...",6,"[Anti Lock Braking System, Centeral Locking, C...",18,[],0,Red,K10B Petrol Engine,2143,74bhp@4000rpm,300Nm@1400-3000rpm,3,4,MPFI,No,No,4299 mm,1780 mm,1433 mm,2380mm,5 Speed,5,Power,Ventilated Disc,Disc,Tubeless Tyres,5


>So far 8397 records obtained out of 9320.  Success Rate-- 90% 

In [169]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8397 entries, 0 to 8396
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   VehicleName          8397 non-null   object
 1   CarLocation          8397 non-null   object
 2   RegistrationYear     8397 non-null   object
 3   Insurance            8397 non-null   object
 4   FuelType             8397 non-null   object
 5   Seats                8397 non-null   object
 6   KmsDriven            8397 non-null   object
 7   RTO                  8397 non-null   object
 8   Ownership            8397 non-null   object
 9   Engine               8397 non-null   object
 10  TransmissionType     8397 non-null   object
 11  YearOfManufacture    8397 non-null   object
 12  VehFeatures          8397 non-null   object
 13  Mileage              8397 non-null   object
 14  MaxPower             8397 non-null   object
 15  Torque               8397 non-null   object
 16  WheelS

### Saving dataset to excel file on local directory

In [167]:
final_df.to_excel(r"C:\Users\hp\Documents\Used_Cars_Complete_data.xlsx")

## Summary-
    -While looping through listing pages(city-links), most pages detected activity and stopped data sharing.
    -So getting data by working separately on each listing pages, more links obtained = 9320
    -Out of 9320 links, 8397 records obtained successfully.
    -Some pages had incomplete data, some cars were sold out, thus redirected to home page website, causing exceptions.
    -Due to power cut-offs & internet disconnectivity, Loop was broke, so had to resume process for remaining links.
    -It took 147 hours runtime i.e, 6 days for complete web scraping process.
    -90% success rate.